This script creates all course-specific predictors for the study sample of all non-first-term observations

In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
vccs_prerequisite_dict = pickle.load(open("~/Box Sync/GAA Transfer/data/intermediate_files/ys8mz/degree_audit_analyses/vccs_prerequisite_dict_2021.p", "rb"))
# vccs_prerequisite_dict['MTH_263'] = [['MTH_167'], ['MTH_161', 'MTH_162'], ['MTH_163', 'MTH_164']]
# vccs_prerequisite_dict['EGR_126'] = [['CSC_110']]

In [3]:
prior_courses = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\full\\prior_courses.dta")
target_courses = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\full\\part1_new.dta")

In [4]:
def create_cdict(df):
    cdict = {}
    for j in range(df.shape[0]):
        course = df.course.iloc[j]
        credit = df.credit.iloc[j]
        est_grade = df.est_grade.iloc[j]
        try:
            cdict[course].append((credit,est_grade))
        except KeyError:
            cdict[course] = [(credit,est_grade)]
    return cdict.copy()

def calc_avg_grade(t):
    tot_cred = 0
    tot_grade_point = 0
    for p in t:
        tot_cred += p[0]
        tot_grade_point += p[0]*p[1]
    return tot_grade_point/tot_cred

In [5]:
vccsid_sets = []
l = list(np.unique(target_courses.vccsid))
n = len(l) // 100 + 1
for i in range(100):
    vccsid_sets.append(l[(n*i):(n*(i+1))])

In [6]:
for i,ll in enumerate(vccsid_sets):
    print(i)
    ll_series = pd.DataFrame({'vccsid':ll})
    prior_part = ll_series.merge(prior_courses, how='inner', on=['vccsid'])
    target_part = ll_series.merge(target_courses, how='inner', on=['vccsid'])
    print(prior_part.shape[0], target_part.shape[0])
    prior_part.to_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\full\\prior_courses\\part{}.dta".format(i),
                        write_index=False)
    target_part.to_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\full\\target_courses\\part{}.dta".format(i),
                        write_index=False)
    print("")

0
28579 10024

1
29215 9755

2
29070 10130

3
28306 9854

4
24734 9865

5
31238 9505

6
29871 9811

7
28706 9981

8
28526 9933

9
29185 9913

10
23571 9891

11
31820 9436

12
29947 9362

13
28504 10141

14
29431 10202

15
29078 9893

16
23536 10031

17
32325 10013

18
31052 9892

19
28272 9653

20
28101 9750

21
28980 9884

22
22749 10403

23
31110 9472

24
31995 9709

25
29041 10196

26
28894 10135

27
29080 10006

28
26553 9993

29
27417 9990

30
32699 9931

31
29558 9662

32
28723 9842

33
28634 9416

34
29016 9894

35
23981 9882

36
32728 9674

37
30238 9447

38
28672 9748

39
28696 9768

40
28749 9995

41
23252 10183

42
32831 9903

43
31245 10038

44
29634 10021

45
28767 9894

46
28022 9536

47
24269 10106

48
30648 9854

49
31858 9976

50
28846 9744

51
28435 9846

52
28518 10129

53
28301 10125

54
26549 9895

55
32570 9698

56
29854 9709

57
29657 10292

58
28304 9864

59
28962 10048

60
23833 10002

61
31848 9707

62
21505 10134

63
12887 10310

64
12616 10155

65
13002 1041

In [6]:
# has_prereq_grade = []
# prereq_grade = []
# for i in range(target_courses.shape[0]):
#     if i % 1000 == 0:
#         print(i)
#     vccsid = target_courses.vccsid.iloc[i]
#     strm = target_courses.strm.iloc[i]
#     course = target_courses.course.iloc[i]
#     prior_sub = prior_courses[prior_courses.vccsid == vccsid]
#     prior_sub = prior_sub[prior_sub.strm < strm]
#     if prior_sub.shape[0] == 0:
#         has_prereq_grade.append(0)
#         prereq_grade.append(0)
#     else:
#         if course not in vccs_prerequisite_dict:
#             has_prereq_grade.append(0)
#             prereq_grade.append(0)
#             continue
#         else:
#             prerequisite_l = []
#             for l in vccs_prerequisite_dict[course]:
#                 prerequisite_l.extend(l)
#             cdict = create_cdict(prior_sub)
#             clist = list(cdict.keys())
#             common_l = np.intersect1d(prerequisite_l, clist)
#             if len(common_l) == 0:
#                 has_prereq_grade.append(0)
#                 prereq_grade.append(0)
#             else:
#                 new_t = []
#                 for c in common_l:
#                     new_t.extend(cdict[c])
#                 has_prereq_grade.append(1)
#                 prereq_grade.append(calc_avg_grade(new_t))

Run the script <b>"parallelized_prereq.py"</b> before the next cell, which uses parallel processing for speeding up.

In [7]:
prereq_r = []
for i in range(100):
    prereq_r.extend(pickle.load(open("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\full\\prereq\\part{}.p".format(i), "rb")))

In [8]:
prereq_df = pd.DataFrame(prereq_r, columns = ['vccsid', 'strm', 'college', 'course', 'section', 'has_prereq_grade', 'prereq_grade'])

In [9]:
print(target_courses.shape[0])
target_courses = target_courses.merge(prereq_df, how='inner', on=['vccsid', 'strm', 'college', 'course', 'section'])
print(target_courses.shape[0])

988799
988799


In [10]:
# has_repeat_grade = []
# repeat_grade = []
# for i in range(target_courses.shape[0]):
#     if i % 1000 == 0:
#         print(i)
#     vccsid = target_courses.vccsid.iloc[i]
#     strm = target_courses.strm.iloc[i]
#     course = target_courses.course.iloc[i]
#     prior_sub = prior_courses[prior_courses.vccsid == vccsid]
#     prior_sub = prior_sub[prior_sub.strm < strm]
#     if prior_sub.shape[0] == 0:
#         has_repeat_grade.append(0)
#         repeat_grade.append(0)
#     else:
#         cdict = create_cdict(prior_sub)
#         if course not in cdict:
#             has_repeat_grade.append(0)
#             repeat_grade.append(0)
#         else:
#             has_repeat_grade.append(1)
#             repeat_grade.append(calc_avg_grade(cdict[course]))

Run the script <b>"parallelized_repeat.py"</b> before the next cell, which uses parallel processing for speeding up.

In [11]:
repeat_r = []
for i in range(100):
    repeat_r.extend(pickle.load(open("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\full\\repeat\\part{}.p".format(i), "rb")))

In [12]:
repeat_df = pd.DataFrame(repeat_r, columns = ['vccsid', 'strm', 'college', 'course', 'section', 'has_repeat_grade', 'repeat_grade'])

In [13]:
print(target_courses.shape[0])
target_courses = target_courses.merge(repeat_df, how='inner', on=['vccsid', 'strm', 'college', 'course', 'section'])
print(target_courses.shape[0])

988799
988799


In [14]:
# target_courses.loc[:,'has_repeat_grade'] = has_repeat_grade
# target_courses.loc[:,'repeat_grade'] = repeat_grade
# target_courses.loc[:,'has_prereq_grade'] = has_prereq_grade
# target_courses.loc[:,'prereq_grade'] = prereq_grade

In [15]:
avg_grades = pd.read_stata("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\full\\avg_grades.dta")
avg_dict = {}
avg_dict_2 = {}
for j in range(avg_grades.shape[0]):
    course = avg_grades.course.iloc[j]
    strm = avg_grades.strm.iloc[j]
    college = avg_grades.college.iloc[j]
    avg_dict[(course,strm,college)] = avg_grades.avg_grade.iloc[j]
    avg_dict_2[(strm,college)] = avg_grades.avg_grade_2.iloc[j]

In [16]:
avg_g = []
for i in range(target_courses.shape[0]):
    course = target_courses.course.iloc[i]
    strm = target_courses.strm.iloc[i]
    college = target_courses.college.iloc[i]
    try:
        avg_g.append(avg_dict[(course,strm,college)])
    except KeyError:
        avg_g.append(avg_dict_2[(strm,college)])
target_courses.loc[:,'avg_g'] = avg_g

In [17]:
target_courses.loc[:,'summer_ind'] = target_courses.strm.apply(lambda x: int(x % 10 == 3))

In [18]:
target_courses.columns.values

array(['vccsid', 'college', 'section', 'strm', 'section_size', 'eve_ind',
       'online_ind', 'lvl2_ind', 'course', 'has_prereq_grade',
       'prereq_grade', 'has_repeat_grade', 'repeat_grade', 'avg_g',
       'summer_ind'], dtype=object)

In [19]:
target_courses = target_courses.loc[:,['vccsid','strm','college','course','section',
                                       'summer_ind', 'section_size', 'eve_ind', 'online_ind', 'lvl2_ind',
                                       'has_repeat_grade', 'repeat_grade', 'has_prereq_grade', 'prereq_grade', 
                                       'avg_g']]
print(target_courses.shape)

(988799, 15)


In [20]:
target_courses.to_csv("C:\\Users\\ys8mz\\Box Sync\\Clickstream\\data\\full\\course_specific_predictors.csv", index=False)